# RFM-анализ

In [1]:
# Импорт библиотек
import pandas as pd
import numpy as np
import psycopg2

In [2]:
# Подключение к БД
conn = psycopg2.connect("dbname='db' user='postgres' password='gfhjkm' host='localhost' port='5432'")

In [3]:
# Функция-запрос в БД
def select_postgresql(sql):
    return pd.read_sql(sql,conn)

In [4]:
# Запрос для извлечения отчета rfm
sql_all_data = '''select * from public.report_rfm_analysis'''

In [5]:
# Считываем данные в датафрейм
df_all_data = select_postgresql(sql_all_data)

In [6]:
# Промежуточный контроль результатов по первым 5 строкам таблицы
df_all_data.head(5)

,customerid,recency,frequency,monetary,rfm_recency,rfm_frequency,rfm_monetary,rfm
0,12346,287,1,77184,3,3,1,331
1,12347,1,6,4085,1,3,2,132
2,12348,37,4,1797,1,3,2,132
3,12350,272,1,334,3,3,3,333
4,12352,34,7,2194,1,3,2,132


### Средний чек по сегменту

In [7]:
# Запрос - количество клиентов в  каждом сегменте, средний чек по сегменту
sql_count_customer_and_avg_check = '''select r.rfm, 
                                             sum(r.monetary) as total_amount,
                                             count(r.rfm) as count_customer,
                                             cast(avg(r.monetary/r.frequency) as integer) as avg_check
                                      from public.report_rfm_analysis as r 
                                      group by r.rfm'''

In [8]:
# Считываем данные в датафрейм
df_count_customer_and_avg_check = select_postgresql(sql_count_customer_and_avg_check)

In [9]:
# Выводим сводную таблицу со средним чеком для каждого сегмента
df_count_customer_and_avg_check

,rfm,total_amount,count_customer,avg_check
0,111,858463,10,1295
1,112,9115,1,128
2,121,1557555,56,1203
3,122,1009709,216,335
4,131,155688,11,2362
5,132,1965101,898,546
6,133,600977,1255,274
7,222,24203,6,358
8,231,75407,3,6329
9,232,294213,154,734


### Структура продаж по странам

In [10]:
# Запрос - структура продаж по странам в абсолютном и относительном выражении
sql_sales_country = '''select d2.rfm,
                              d2.country,
                              cast(sum(d2.amount) as integer) as amount_country,
                              round(cast(sum(d2.amount)/sum(sum(d2.amount))over(partition by d2.rfm)*100 as numeric),2) as percent_total_amount
                       from 
                           (select d.*, d.quantity * d.unitprice as amount, r.rfm 
                            from public.dataset as d left join public.report_rfm_analysis as r on d.customerid = r.customerid 
                            where d.invoicedate < date('2011-11-01')) as d2
                       group by d2.rfm, d2.country
                       order by d2.rfm, sum(d2.amount)desc'''

In [11]:
# Считываем данные в датафрейм
df_sales_country = select_postgresql(sql_sales_country)

In [12]:
# Выводим сводную таблицу по странам
df_sales_country.head(10)

,rfm,country,amount_country,percent_total_amount
0,111,United Kingdom,501234,58.39
1,111,Netherlands,243103,28.32
2,111,EIRE,114125,13.29
3,112,United Kingdom,9115,100.00
4,121,United Kingdom,1213646,77.92
5,121,Australia,118560,7.61
6,121,EIRE,110627,7.10
7,121,France,46296,2.97
8,121,Sweden,30430,1.95
9,121,Germany,26057,1.67


### Топ-3 дня по объему продаж

In [13]:
# Запрос - Топ-3 дня по объему продаж в разрезе сегмент-страна
sql_top_3_days_sales = '''select d4.rfm, d4.country, max(d4.top) as top_3_days
                          from 
                              (select d3.rfm, d3.country, string_agg(d3.day_of_week,', ')over(partition by d3.rfm, d3.country) as top
                              from 
                                  (select d2.rfm, d2.country, d2.day_of_week,sum(d2.amount) as total_amount,
                                          row_number ()over(partition by d2.rfm, d2.country order by d2.rfm, d2.country, sum(d2.amount)desc)
                                  from 
                                      (select r.rfm,  d.country,func_day_of_week(cast(to_char(d.invoicedate, 'D') as integer)) as day_of_week,
                                              d.quantity * d.unitprice as amount
                                       from public.dataset as d left join public.report_rfm_analysis as r on d.customerid = r.customerid
                                        where d.invoicedate < date('2011-11-01')) as d2
                                  group by d2.rfm, d2.country, d2.day_of_week
                                  order by d2.rfm, d2.country, sum(d2.amount) desc) as d3
                              where d3.row_number <= 3) as d4
                           group by d4.rfm, d4.country'''

In [14]:
# Считываем данные в датафрейм
df_top_3_days_sales = select_postgresql(sql_top_3_days_sales)

In [15]:
# Выводим сводную таблицу по дням недели
df_top_3_days_sales.head(10)

,rfm,country,top_3_days
0,111,EIRE,"thursday, friday, monday"
1,111,Netherlands,"thursday, tuesday, monday"
2,111,United Kingdom,"tuesday, thursday, monday"
3,112,United Kingdom,"wednesday, thursday, tuesday"
4,121,Australia,"thursday, wednesday, tuesday"
5,121,EIRE,"friday, tuesday, monday"
6,121,France,"friday, thursday, monday"
7,121,Germany,"friday, wednesday, monday"
8,121,Spain,"tuesday, monday, thursday"
9,121,Sweden,"monday, friday, tuesday"
